In [1]:
# Import the numpy and pandas package
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
df = pd.read_excel("./data/Company_data.xlsx")
df.columns

Index(['(x(i))', '(y(i))'], dtype='object')

In [3]:
# Defining the X and the y variables
X = df['(x(i))']
y = df['(y(i))']

In [4]:
# Basic calculations
n = len(X)          # Defining the total number of observations within the dataset
p = 2               # (2) Features have will be estimated in simple linear regression that are b-0 and b-1        
# -------------------------------
x_avg = X.mean()
y_avg = y.mean()
# -------------------------------
# Defining a new column to hold the results of the multiplication of the two columns (x(i)) & (y(i)) 
df['(x(i)) * (y(i))'] = df['(x(i))'] * df['(y(i))']

# Defining a new column to hold the results of the (x(i)^2) 
df['(x(i)^2)'] = np.power(df['(x(i))'],2)
# -------------------------------
df.head()

,(x(i)),(y(i)),(x(i)) * (y(i)),(x(i)^2)
0,230.1,22.1,5085.21,52946.01
1,44.5,10.4,462.80,1980.25
2,17.2,12.0,206.40,295.84
3,151.5,16.5,2499.75,22952.25
4,180.8,17.9,3236.32,32688.64


In [5]:
# Calculating the "Simple Linear Regression" parametrs (b-0 & b-1)
 
b_1 = (np.sum(df['(x(i)) * (y(i))']) - (n * x_avg * y_avg)) / (np.sum(df['(x(i)^2)']) - (n * np.power(x_avg, 2))) 

b_0 = y_avg - (b_1 * x_avg)

# Printing the parametrs values 
print(f"b_1: {b_1}")
print(f"b_0: {b_0}")

b_1: 0.055464770469558736
b_0: 6.9748214882299155


In [6]:
# The estiamted y_values and the error terms 

# Calculating the predicted value of the response variable based on the simple linear regression equation
df['(y(i)_predicted)'] = b_0 + (b_1 * df['(x(i))'] ) 

# Calculating the error terms, which are the difference between the actual y_values and the predicted y_values
df['(y(i)) - (y(i)_predicted)']=  df['(y(i))'] - df['(y(i)_predicted)']
# Notice that '(y(i)) - (y(i)_predicted)' is the same as 'e(i)'

# Calculating the squred-value for each of the error terms, and holding that in a new column in the dataframe
df['e(i)^2'] =np.power(df['(y(i)) - (y(i)_predicted)'], 2)


In [7]:
# The varience of the residuals (σe)^2 
var_residuals = (np.sum(df['e(i)^2']) / (n - p))   # The varience of the residuals is the same as "MSE"
var_residuals  # ((σe)^2 )

5.27044838161124

In [8]:
# Calculating the varience of betas

# First computing the quantity of sxx which is ∑ i = 1 n (( X i − X ¯ )^2). 
df ['(x(i)) - x_avg)']  = df['(x(i))'] - x_avg 
df ['(x(i)) - x_avg)^2'] = np.power(df ['(x(i)) - x_avg)'], 2)

sxx = np.sum(df ['(x(i)) - x_avg)^2'])

# -------------------------------

var_b_0 = (var_residuals / n) + (var_residuals * ((np.power(x_avg,2))/sxx))

print(f'var_b_0 = {var_b_0}')

# -------------------------------

var_b_1 = var_residuals / sxx

print(f'var_b_1 = {var_b_1}')

var_b_0 = 0.10404075059042824
var_b_1 = 3.5931142685697453e-06


In [9]:
# "Summary Table" of coefficients

beta = np.array([b_0 , b_1])
std_error_beta = np.array([np.sqrt(var_b_0), np.sqrt(var_b_1)])
t_values_beta = np.divide(beta, std_error_beta) # Calculating the t-values for each coefficient. 
# The t-value is a measure of how many standard errors the estimated coefficient is away from zero.

# Calcualting the P-value for the betas
# Degrees of freedom (df) based on the number of samples and parameters in your model
dof = n - p  # The degree of freedom is the total number of observations - 2

# Calculate the p-values
p_values = 2 * (1 - stats.t.cdf(np.abs(t_values_beta), dof))
# print(p_values)
# -------------------------------
# Define the confidence level (1 - alpha)
alpha = 0.05  # For a 95% confidence level, alpha is 0.05

# Calculate the critical value from the t-distribution
critical_value = stats.t.ppf( 1 - alpha / 2, dof)  # For a two-tailed test

# std_error_beta, beta, and p_values are defined 
margin_of_error = critical_value * std_error_beta

# Calculate the lower and upper bounds of the confidence intervals
ci_lower = beta - margin_of_error
ci_upper = beta + margin_of_error

# Create a DataFrame with your data
summary_beta = {'Beta': beta, 
                'Std. Error': std_error_beta, 
                't-value': t_values_beta, 
                'P-value': p_values, 
                 f'{int((1-alpha)*100)}% CI Lower': ci_lower, 
                 f'{int((1-alpha)*100)}% CI upper': ci_upper}

# Create a DataFrame from the summary_beta
coef_summary_df = pd.DataFrame(summary_beta) 
        
# Renaming the indices 
coef_summary_df  = coef_summary_df .rename(index={i: f'beta-{i}' for i in range(len(coef_summary_df ))})

# Print the resulting DataFrame
print(coef_summary_df)

            Beta  Std. Error    t-value  P-value  95% CI Lower  95% CI upper
beta-0  6.974821    0.322553  21.623767      0.0      6.338740      7.610903
beta-1  0.055465    0.001896  29.260497      0.0      0.051727      0.059203


In [10]:
# Calculating "Sum of Squares" 

# -------------------------------
# Calculate SST, MST
df ['(y(i)) - y_avg)']  = df['(y(i))'] - y_avg 
df ['(y(i)) - y_avg)^2'] = np.power(df ['(y(i)) - y_avg)'], 2)
SST = np.sum(df ['(y(i)) - y_avg)^2'])
MST = SST / (n - 1)
df
# Calculate SSE, MSE
SSE = np.sum(df['e(i)^2'])
MSE = SSE / (n - p)    # p = 2 (Simple Linear regression)

# Calculate SSR, MSR
df ['(y(i)_predicted) - y_avg)']  = df['(y(i)_predicted)'] - y_avg 
df ['(y(i)_predicted) - y_avg)^2'] = np.power(df ['(y(i)_predicted) - y_avg)'], 2)
SSR = np.sum(df ['(y(i)_predicted) - y_avg)^2'])
MSR = SSR / (p-1)
# -------------------------------
# Calculate the F-statistic
F_statistic = MSR / MSE

# Calculate the p-value for regression
dof_numerator = p-1  # Degrees of freedom for the numerator (Regression)
dof_denominator = n - p  # Degrees of freedom for the denominator (Error)

# Calculate the p-value
p_value = 1 - stats.f.cdf(F_statistic, dof_numerator, dof_denominator)

# Create a dictionary with the calculated values
data = {
    'Source of Variation': ['Total', 'Regression', 'Error'],
    'Sum of Squares (SS)': [SST, SSR, SSE],
    'Degrees of Freedom (df)': [n - 1, p-1, n - p ],
    'Mean Square (MS)': [MST, MSR, MSE],
    'F Value': [MST/MSE, MSR/MSE, None],
    'p Value' : [None, p_value, None]
}

# Create a DataFrame from the dictionary
anova_table = pd.DataFrame(data)

# Set the 'Source of Variation' column as the index
anova_table.set_index('Source of Variation', inplace=True)

# Print the resulting ANOVA table
print(anova_table)

# NOTE: THE MOST IMPORTANT F-VALUE IF THE ONE FOR THE REGRESSION

                     Sum of Squares (SS)  Degrees of Freedom (df)  \
Source of Variation                                                 
Total                         5555.98395                      199   
Regression                    4512.43517                        1   
Error                         1043.54878                      198   

                     Mean Square (MS)     F Value       p Value  
Source of Variation                                              
Total                       27.919517    5.297370           NaN  
Regression                4512.435170  856.176713  1.110223e-16  
Error                        5.270448         NaN           NaN  


In [11]:
# Adding some metrics
# Calculating the R^2
R_squared = 1 - (SSE / SST)
print(f"R-squared: {R_squared}")

# Calculating the R^2 adjusted
R_squared_adjusted = 1 - (MSE / MST)
print(f"Adjusted R-squared: {R_squared_adjusted}")

# The amount of variance reduction with the designed model is 81.22% compared to the null model (the mean
# value).
# The learned variability using the model is equal to the 81.22%, the remaining unknown variability is 18.78%.

R-squared: 0.8121757029987414
Adjusted R-squared: 0.8112270954381291


In [12]:
# Confidence Intervals 

# Constructing a confidence interval around the mean response E(y) = µ(y) given the predictor value x(h)

# Degrees of freedom (df) based on the number of samples and parameters in your model
dof = n - p   # In other words the degree of freedom is the total number of observations - 2 

# Define the confidence level (1 - alpha)
alpha = 0.05  # For a 95% confidence level, alpha is 0.05

# Calculate the critical value from the t-distribution
critical_value = stats.t.ppf( 1 - alpha / 2, dof)  # For a two-tailed test

# #  sxx which is ∑ i = 1 n (( X i − X ¯ )^2) has already been calculated. 
# df ['(x(i)) - x_avg)']  = df['(x(i))'] - x_avg 
# df ['(x(i)) - x_avg)^2'] = np.power(df ['(x(i)) - x_avg)'], 2)

# sxx = np.sum(df ['(x(i)) - x_avg)^2'])

df['subformula_c'] = (( 1 / n) + (np.power((df['(x(i))'] - x_avg),2) / sxx))
df['standard_error_of_fitting_c'] = critical_value*  np.sqrt(var_residuals * df['subformula_c'])

# -------------------------------


# Calculate the lower and upper bounds of the confidence intervals
df['ci_lower'] =  df['(y(i)_predicted)'] - df['standard_error_of_fitting_c']
df['ci_upper'] =  df['(y(i)_predicted)'] + df['standard_error_of_fitting_c']
df.head()

,(x(i)),(y(i)),(x(i)) * (y(i)),(x(i)^2),(y(i)_predicted),(y(i)) - (y(i)_predicted),e(i)^2,(x(i)) - x_avg),(x(i)) - x_avg)^2,(y(i)) - y_avg),(y(i)) - y_avg)^2,(y(i)_predicted) - y_avg),(y(i)_predicted) - y_avg)^2,subformula_c,standard_error_of_fitting_c,ci_lower,ci_upper
0,230.1,22.1,5085.21,52946.01,19.737265,2.362735,5.582516,83.0575,6898.548306,6.9695,48.57393,4.606765,21.222285,0.009703,0.445953,19.291312,20.183218
1,44.5,10.4,462.80,1980.25,9.443004,0.956996,0.915842,-102.5425,10514.964306,-4.7305,22.37763,-5.687496,32.347613,0.012169,0.499406,8.943598,9.942410
2,17.2,12.0,206.40,295.84,7.928816,4.071184,16.574543,-129.8425,16859.074806,-3.1305,9.80003,-7.201684,51.864259,0.016494,0.581424,7.347392,8.510239
3,151.5,16.5,2499.75,22952.25,15.377734,1.122266,1.259480,4.4575,19.869306,1.3695,1.87553,0.247234,0.061125,0.005014,0.320558,15.057176,15.698293
4,180.8,17.9,3236.32,32688.64,17.002852,0.897148,0.804875,33.7575,1139.568806,2.7695,7.67013,1.872352,3.505702,0.005777,0.344098,16.658754,17.346950


In [13]:
# Prediction Intervals 
df['subformula_pr'] = (1 + ( 1 / n) + (np.power((df['(x(i))'] - x_avg),2) / sxx))
df['standard_error_of_fitting_pr'] = critical_value*  np.sqrt(var_residuals * df['subformula_pr'])

# Calculate the lower and upper bounds of the prediction intervals
df['pi_lower'] =  df['(y(i)_predicted)'] - df['standard_error_of_fitting_pr']
df['pi_upper'] =  df['(y(i)_predicted)'] + df['standard_error_of_fitting_pr']
df.head()

,(x(i)),(y(i)),(x(i)) * (y(i)),(x(i)^2),(y(i)_predicted),(y(i)) - (y(i)_predicted),e(i)^2,(x(i)) - x_avg),(x(i)) - x_avg)^2,(y(i)) - y_avg),...,(y(i)_predicted) - y_avg),(y(i)_predicted) - y_avg)^2,subformula_c,standard_error_of_fitting_c,ci_lower,ci_upper,subformula_pr,standard_error_of_fitting_pr,pi_lower,pi_upper
0,230.1,22.1,5085.21,52946.01,19.737265,2.362735,5.582516,83.0575,6898.548306,6.9695,...,4.606765,21.222285,0.009703,0.445953,19.291312,20.183218,1.009703,4.549162,15.188103,24.286427
1,44.5,10.4,462.80,1980.25,9.443004,0.956996,0.915842,-102.5425,10514.964306,-4.7305,...,-5.687496,32.347613,0.012169,0.499406,8.943598,9.942410,1.012169,4.554712,4.888291,13.997716
2,17.2,12.0,206.40,295.84,7.928816,4.071184,16.574543,-129.8425,16859.074806,-3.1305,...,-7.201684,51.864259,0.016494,0.581424,7.347392,8.510239,1.016494,4.564433,3.364382,12.493249
3,151.5,16.5,2499.75,22952.25,15.377734,1.122266,1.259480,4.4575,19.869306,1.3695,...,0.247234,0.061125,0.005014,0.320558,15.057176,15.698293,1.005014,4.538585,10.839149,19.916319
4,180.8,17.9,3236.32,32688.64,17.002852,0.897148,0.804875,33.7575,1139.568806,2.7695,...,1.872352,3.505702,0.005777,0.344098,16.658754,17.346950,1.005777,4.540309,12.462543,21.543161


In [14]:
# For a new data-point
x_newpoint = 50

y_predicted_newpoint = b_0 + (b_1 * x_newpoint)
print(f"Predicted y for x = {x_newpoint}: {y_predicted_newpoint}")

prediction_subformula = (1 + (1 / n) + (np.power((x_newpoint - x_avg), 2) / sxx))
std_error_prediction = critical_value * np.sqrt(var_residuals * prediction_subformula)

lower_prediction = y_predicted_newpoint - std_error_prediction
upper_prediction = y_predicted_newpoint + std_error_prediction

print(f"Standard Error of Prediction: {std_error_prediction}")
print(f"Lower Prediction Bound: {lower_prediction}")
print(f"Upper Prediction Bound: {upper_prediction}")


Predicted y for x = 50: 9.748060011707853
Standard Error of Prediction: 4.553028300453592
Lower Prediction Bound: 5.195031711254261
Upper Prediction Bound: 14.301088312161445
